In [44]:
#import all libraries needed

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV


In [45]:
from sklearn.datasets import fetch_20newsgroups

# Fetch the training data
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
# newsgroups_train = fetch_20newsgroups(subset='train')
# Fetch the test data
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))
# newsgroups_test = fetch_20newsgroups(subset='test')
print("attributes for data:",dir(newsgroups_train))
print("attributes for test:",dir(newsgroups_test))


attributes for data: ['DESCR', 'data', 'filenames', 'target', 'target_names']
attributes for test: ['DESCR', 'data', 'filenames', 'target', 'target_names']


In [46]:
len_train = len(newsgroups_train.data)
len_test = len(newsgroups_test.data)
print("# of training data:",len_train)
print("# of testing data:", len_test)  

train_list_category = []
for target_names in newsgroups_train.target_names and newsgroups_test.target_names:
    train_list_category.append(target_names)
    
        
print("category:",train_list_category)    
print("length of category:",len(train_list_category))

# print(newsgroups_train.data[20000])  # print the first document
# print("\nClass label:", newsgroups_train.target[20000])  # print its associated label
# print("\nClass name:", newsgroups_train.target_names[newsgroups_train.target[0]])  # print the name of the class

# of training data: 11314
# of testing data: 7532
category: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
length of category: 20


In [47]:
#Data Preprocessing

from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000) # stop words are removed like and and the

# Fit and transform the training data
X_train = vectorizer.fit_transform(newsgroups_train.data)

# Transform the test data
X_test = vectorizer.transform(newsgroups_test.data)

# Target labels
y_train = newsgroups_train.target
y_test = newsgroups_test.target


In [48]:
# Regression model
from sklearn.linear_model import LogisticRegression

# Create a logistic regression model
logreg = LogisticRegression(max_iter=1000)

# Train the model
logreg.fit(X_train, y_train)

# Score on training and test data
logreg_train_score = logreg.score(X_train, y_train)
logreg_test_score = logreg.score(X_test, y_test)

print(f"Logistic Regression - Training accuracy: {logreg_train_score:.4f}")
print(f"Logistic Regression - Test accuracy: {logreg_test_score:.4f}")


Logistic Regression - Training accuracy: 0.8611
Logistic Regression - Test accuracy: 0.6471


In [49]:
# Create a decision tree classifier
from sklearn.tree import DecisionTreeClassifier

# Create a decision tree classifier
dtree = DecisionTreeClassifier()

# Train the model
dtree.fit(X_train, y_train)

# Score on training and test data
dtree_train_score = dtree.score(X_train, y_train)
dtree_test_score = dtree.score(X_test, y_test)

print(f"Decision Tree - Training accuracy: {dtree_train_score:.4f}")
print(f"Decision Tree - Test accuracy: {dtree_test_score:.4f}")


Decision Tree - Training accuracy: 0.9712
Decision Tree - Test accuracy: 0.4365


In [50]:
#Support Vector Machines (LinearSVC)
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_test = vectorizer.transform(newsgroups_test.data)
y_train = newsgroups_train.target
y_test = newsgroups_test.target

from sklearn.svm import LinearSVC

svc_model = LinearSVC(dual=False)
svc_model.fit(X_train, y_train)

train_accuracy = svc_model.score(X_train, y_train)
test_accuracy = svc_model.score(X_test, y_test)

print(f"LinearSVC - Training accuracy: {train_accuracy:.4f}")
print(f"LinearSVC - Test accuracy: {test_accuracy:.4f}")


LinearSVC - Training accuracy: 0.9725
LinearSVC - Test accuracy: 0.6918


In [53]:
#AdaBoost
from sklearn.ensemble import AdaBoostClassifier

# Initialize AdaBoost classifier
ada_clf = AdaBoostClassifier(n_estimators=50, random_state=0)

# Train AdaBoost
ada_clf.fit(X_train, y_train)

# Get training and test accuracies
train_accuracy = ada_clf.score(X_train, y_train)
test_accuracy = ada_clf.score(X_test, y_test)

print("AdaBoost - Training accuracy:", round(train_accuracy, 4))
print("AdaBoost - Test accuracy:", round(test_accuracy, 4))


AdaBoost - Training accuracy: 0.4167
AdaBoost - Test accuracy: 0.3745


In [54]:

from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)

# Train the classifier
rf_clf.fit(X_train, y_train)

# Get training and test accuracies
train_accuracy_rf = rf_clf.score(X_train, y_train)
test_accuracy_rf = rf_clf.score(X_test, y_test)

print(f"Random Forest - Training accuracy: {train_accuracy_rf:.4f}")
print(f"Random Forest - Test accuracy: {test_accuracy_rf:.4f}")


Random Forest - Training accuracy: 0.9747
Random Forest - Test accuracy: 0.5928


In [55]:
#Gradient Boosting

from sklearn.ensemble import GradientBoostingClassifier

# Initialize the Gradient Boosting classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

# Train the Gradient Boosting classifier
gb_clf.fit(X_train, y_train)

# Calculate training accuracy
train_accuracy = gb_clf.score(X_train, y_train)
print(f"Gradient Boosting - Training accuracy: {train_accuracy:.4f}")

# Calculate test accuracy
test_accuracy = gb_clf.score(X_test, y_test)
print(f"Gradient Boosting - Test accuracy: {test_accuracy:.4f}")


Gradient Boosting - Training accuracy: 0.3641
Gradient Boosting - Test accuracy: 0.3031
